## 2. **Implement the Bayes classifier over the breast cancer wisconsin dataset.**

In [8]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import math


#Breas Cancer dataset loaded
data = load_breast_cancer()
dataset = pd.DataFrame(data.data, columns=data.feature_names)
dataset['target'] = data.target


x = dataset.iloc[:, :-1].to_numpy()
y = dataset.iloc[:, -1].to_numpy()

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=1)


#class created for bayes classifier and its related functions
class BayesClassifier(object):


  def __init__(self,x,y):
    self.classes = self.separateByClass(x,y)
    self.meanS = self.meanStd(self.classes)
    self.probClass = self.classP(y)


  def meanStd(self,classes):
    meanStd = {}
    for (key, value) in classes.items():
      meanStd[key] = [(np.mean(attr), np.std(attr)) for attr in zip(*value)]
    return meanStd

  #Split dataset by class and calculation for each row
  def separateByClass(self,x,y):
    classes = {}
    for i in range(len(x)):
      row = x[i]
      if y[i] not in classes:
        classes[y[i]] = []
      classes[y[i]].append(row)
    return classes

  def classP(self,y):
    classProb = {}
    classProb[1] = list(y).count(1)/len(y) *100
    classProb[0] = list(y).count(0)/len(y) *100
    return classProb

  def P(self,meanStd,classProb,x):
    probs = {}
    for (key, value) in meanStd.items():
      probs[key] = 1
      for i in range((len(value))):
        (mean,std) = value[i]
        probs[key] *= self.gaussianProbFn(x[i],mean,std)
      probs[key] *= self.probClass[key]
    return probs

  #Probability Distribution Function
  def gaussianProbFn(self,x,mean,std):

    if std == 0.0:
      if x == mean:
        return 1.0
      else:
        return 0.0

    else:
      exp = math.exp(-(np.power(x - mean, 2) / (2 * np.power(std, 2))))
      return 1 / (math.sqrt(2 * math.pi) * std) * exp

  def pred(self,x):
    predictions = []
    for i in range(len(x)):
      probs = self.P(self.meanS,self.probClass, x_test[i])
      bestP=-1
      bestClass=None
      for (key,value) in probs.items():
        if bestClass is None or value > bestP:
          bestP = value
          bestClass = key
      predictions.append(bestClass)
    return predictions

  def accuracy(self,realTarget,predTarget):
    c=0
    for i in range(len(realTarget)):
      if realTarget[i]==predTarget[i]:
        c+=1
    return c/float(len(realTarget))*100.0


bayesClass=BayesClassifier(x_train,y_train)

meanStd=bayesClass.meanStd

accuracy = bayesClass.accuracy(y_test,bayesClass.pred(x_test))


print("Accuracy using Bayes Classifier : ",accuracy)

Accuracy using Bayes Classifier :  94.4055944055944
